# KILT-3-UniformRandomSampling

- Uniform Random Sampling for negative ctxs

In [ ]:
# FiD_nq_dpr 
# /data/philhoon-relevance/FiD/open_domain_data/NQ
# /data/philhoon-relevance/FiD/open_domain_data/NQ/train.json
# /data/philhoon-relevance/FiD/open_domain_data/NQ/dev.json
# /data/philhoon-relevance/FiD/open_domain_data/NQ/test.json

# kilt_nq_dpr
# /data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-train-multikilt.json
# /data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-dev-multikilt.json

### MogoDB CHECK

In [35]:
from pprint import pprint
import random
from copy import deepcopy
from util import utils
from copy import deepcopy

In [10]:
SEED = 42
utils.seed_everything(42)

In [11]:
from kilt.knowledge_source import KnowledgeSource

In [12]:
# get the knowledge souce
ks = KnowledgeSource()

In [13]:
# count entries - 5903530
total_length = ks.get_num_pages()

In [14]:
total_length

5903530

In [15]:
# get page by id
page_id = ks.get_page_by_id(27097632)

In [16]:
# pprint(page_id['text'][8])

In [17]:
distinct_ids = ks.get_distinct_ids()

In [18]:
len(distinct_ids)

5903530

In [19]:
type(distinct_ids)

set

In [20]:
for i in distinct_ids:
    print(type(i))
    break

<class 'str'>


In [21]:
# get pages by title
page = ks.get_page_by_title("Michael Jordan")

In [23]:
# pprint(page)

### Get kilt_dpr_nq_dev_pos1.json
    - In order to get the same quetions
    - 1st ctx (0 in index) is the positive ctx

In [24]:
filename = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25/kilt_bm25_nq_dev_pos1.json'

In [25]:
kilt_bm25_nq_dev_pos1 = utils.open_json(filename)

In [26]:
pprint(kilt_bm25_nq_dev_pos1[0])

{'answers': ['the therefore sign',
             'therefore sign',
             'the therefore sign ( ∴ ) is generally used before a logical '
             'consequence , such as the conclusion of a syllogism',
             'a logical consequence , such as the conclusion of a syllogism'],
 'ctxs': [{'psg_id': 3894652,
           'text': 'Therefore sign In logical argument and mathematical proof, '
                   'the therefore sign (∴) is generally used before a logical '
                   'consequence, such as the conclusion of a syllogism. The '
                   'symbol consists of three dots placed in an upright '
                   'triangle and is read "therefore". It is encoded at . While '
                   'it is not generally used in formal writing, it is used in '
                   'mathematics and shorthand. It is complementary to . '
                   'History. According to Cajori, "A History of Mathematical '
                   'Notations," Johann Rahn used both t

In [28]:
# Checking dpr format does not include passage more than 100-word chunks
for instance in kilt_bm25_nq_dev_pos1:
    for ctx in instance['ctxs']:
        text_input = ctx['text']
        if len(text_input.split(' ')) > 100:
            pprint(ctx)
            break

### Git KILT_nq_dev
- to get wikipedia id of provenance.
- how to match? using question

In [29]:
import jsonlines

In [30]:
filename_2 = '/data/yorih-deardr/data/kilt/nq-dev-kilt.jsonl'

In [31]:
def open_jsonl(file):
    json_lst = []
    with jsonlines.open(file , 'r') as f: 
        for line in f:
            json_lst.append(line)
    return json_lst
        

In [32]:
kilt_nq_dev = open_jsonl(filename_2)

In [33]:
# kilt_nq_dev[0]

{'id': '6915606477668963399',
 'input': 'what do the 3 dots mean in math',
 'output': [{'answer': 'the therefore sign',
   'provenance': [{'wikipedia_id': '10593264',
     'title': 'Therefore sign',
     'start_paragraph_id': 1,
     'start_character': 44,
     'end_paragraph_id': 1,
     'end_character': 62,
     'bleu_score': 1.0,
     'section': 'Section::::Abstract.'}]},
  {'answer': 'therefore sign',
   'provenance': [{'wikipedia_id': '10593264',
     'title': 'Therefore sign',
     'start_paragraph_id': 1,
     'start_character': 48,
     'end_paragraph_id': 1,
     'end_character': 62,
     'bleu_score': 1.0,
     'section': 'Section::::Abstract.'}]},
  {'answer': 'a logical consequence , such as the conclusion of a syllogism'},
  {'answer': 'the therefore sign ( ∴ ) is generally used before a logical consequence , such as the conclusion of a syllogism'},
  {'provenance': [{'wikipedia_id': '10593264',
     'title': 'Therefore sign',
     'start_paragraph_id': 1,
     'start_char

In [36]:
def get_random_samples(distinct_ids, wiki_ids, ks):
    flag = 1
    cand_ctx = []
    all_but_wiki_ids = distinct_ids.difference(wiki_ids)
    
    while flag:
        random_ids = random.sample (all_but_wiki_ids, 20)
        for r_id in random_ids:
            random_page = ks.get_page_by_id(r_id)
            if 'wikipedia_title' in random_page and len(random_page['wikipedia_title']) > 2:
                title_ = random_page['wikipedia_title']
                for text_ in random_page['text']:
                    if 'BULLET' not in text_ and 'Section' not in text_ and len(text_) > 100 and len(text_.split(' ')) < 100:
                        template = {
                            'text' : text_,
                            'title': title_
                        }
                        cand_ctx.append(template)
                        break
        if len(cand_ctx) >= 10:
            flag = 0
            
    random_samples = random.sample(cand_ctx, 5)
    
    return random_samples

In [38]:
kilt_bm25_nq_dev_pos = deepcopy(kilt_bm25_nq_dev_pos1)

In [40]:
# pprint(kilt_bm25_nq_dev_pos[0])

In [42]:
i = 0
j = 0

while i<len(kilt_bm25_nq_dev_pos):

    if kilt_bm25_nq_dev_pos[i]['question'] == kilt_nq_dev[j]['input']:
        wiki_ids = set([])
        for answer_format in kilt_nq_dev[j]['output']:
            if 'provenance' in answer_format:
                for provenance_format in answer_format['provenance']:
                    if 'wikipedia_id' in provenance_format:
                        wiki_ids.add(provenance_format['wikipedia_id'])
        
        random_samples = get_random_samples(distinct_ids, wiki_ids, ks)
        
        random_samples[0] = kilt_bm25_nq_dev_pos[i]['ctxs'][0]
        kilt_bm25_nq_dev_pos[i]['ctxs'] = random_samples
        
        i += 1
        j += 1
        print(f'total : {i}th of {len(kilt_bm25_nq_dev_pos)}')
    else:
        j += 1 
    


total : 1th of 2539
total : 2th of 2539
total : 3th of 2539
total : 4th of 2539
total : 5th of 2539
total : 6th of 2539
total : 7th of 2539
total : 8th of 2539
total : 9th of 2539
total : 10th of 2539
total : 11th of 2539
total : 12th of 2539
total : 13th of 2539
total : 14th of 2539
total : 15th of 2539
total : 16th of 2539
total : 17th of 2539
total : 18th of 2539
total : 19th of 2539
total : 20th of 2539
total : 21th of 2539
total : 22th of 2539
total : 23th of 2539
total : 24th of 2539
total : 25th of 2539
total : 26th of 2539
total : 27th of 2539
total : 28th of 2539
total : 29th of 2539
total : 30th of 2539
total : 31th of 2539
total : 32th of 2539
total : 33th of 2539
total : 34th of 2539
total : 35th of 2539
total : 36th of 2539
total : 37th of 2539
total : 38th of 2539
total : 39th of 2539
total : 40th of 2539
total : 41th of 2539
total : 42th of 2539
total : 43th of 2539
total : 44th of 2539
total : 45th of 2539
total : 46th of 2539
total : 47th of 2539
total : 48th of 2539
t

total : 380th of 2539
total : 381th of 2539
total : 382th of 2539
total : 383th of 2539
total : 384th of 2539
total : 385th of 2539
total : 386th of 2539
total : 387th of 2539
total : 388th of 2539
total : 389th of 2539
total : 390th of 2539
total : 391th of 2539
total : 392th of 2539
total : 393th of 2539
total : 394th of 2539
total : 395th of 2539
total : 396th of 2539
total : 397th of 2539
total : 398th of 2539
total : 399th of 2539
total : 400th of 2539
total : 401th of 2539
total : 402th of 2539
total : 403th of 2539
total : 404th of 2539
total : 405th of 2539
total : 406th of 2539
total : 407th of 2539
total : 408th of 2539
total : 409th of 2539
total : 410th of 2539
total : 411th of 2539
total : 412th of 2539
total : 413th of 2539
total : 414th of 2539
total : 415th of 2539
total : 416th of 2539
total : 417th of 2539
total : 418th of 2539
total : 419th of 2539
total : 420th of 2539
total : 421th of 2539
total : 422th of 2539
total : 423th of 2539
total : 424th of 2539
total : 42

total : 754th of 2539
total : 755th of 2539
total : 756th of 2539
total : 757th of 2539
total : 758th of 2539
total : 759th of 2539
total : 760th of 2539
total : 761th of 2539
total : 762th of 2539
total : 763th of 2539
total : 764th of 2539
total : 765th of 2539
total : 766th of 2539
total : 767th of 2539
total : 768th of 2539
total : 769th of 2539
total : 770th of 2539
total : 771th of 2539
total : 772th of 2539
total : 773th of 2539
total : 774th of 2539
total : 775th of 2539
total : 776th of 2539
total : 777th of 2539
total : 778th of 2539
total : 779th of 2539
total : 780th of 2539
total : 781th of 2539
total : 782th of 2539
total : 783th of 2539
total : 784th of 2539
total : 785th of 2539
total : 786th of 2539
total : 787th of 2539
total : 788th of 2539
total : 789th of 2539
total : 790th of 2539
total : 791th of 2539
total : 792th of 2539
total : 793th of 2539
total : 794th of 2539
total : 795th of 2539
total : 796th of 2539
total : 797th of 2539
total : 798th of 2539
total : 79

total : 1122th of 2539
total : 1123th of 2539
total : 1124th of 2539
total : 1125th of 2539
total : 1126th of 2539
total : 1127th of 2539
total : 1128th of 2539
total : 1129th of 2539
total : 1130th of 2539
total : 1131th of 2539
total : 1132th of 2539
total : 1133th of 2539
total : 1134th of 2539
total : 1135th of 2539
total : 1136th of 2539
total : 1137th of 2539
total : 1138th of 2539
total : 1139th of 2539
total : 1140th of 2539
total : 1141th of 2539
total : 1142th of 2539
total : 1143th of 2539
total : 1144th of 2539
total : 1145th of 2539
total : 1146th of 2539
total : 1147th of 2539
total : 1148th of 2539
total : 1149th of 2539
total : 1150th of 2539
total : 1151th of 2539
total : 1152th of 2539
total : 1153th of 2539
total : 1154th of 2539
total : 1155th of 2539
total : 1156th of 2539
total : 1157th of 2539
total : 1158th of 2539
total : 1159th of 2539
total : 1160th of 2539
total : 1161th of 2539
total : 1162th of 2539
total : 1163th of 2539
total : 1164th of 2539
total : 116

total : 1479th of 2539
total : 1480th of 2539
total : 1481th of 2539
total : 1482th of 2539
total : 1483th of 2539
total : 1484th of 2539
total : 1485th of 2539
total : 1486th of 2539
total : 1487th of 2539
total : 1488th of 2539
total : 1489th of 2539
total : 1490th of 2539
total : 1491th of 2539
total : 1492th of 2539
total : 1493th of 2539
total : 1494th of 2539
total : 1495th of 2539
total : 1496th of 2539
total : 1497th of 2539
total : 1498th of 2539
total : 1499th of 2539
total : 1500th of 2539
total : 1501th of 2539
total : 1502th of 2539
total : 1503th of 2539
total : 1504th of 2539
total : 1505th of 2539
total : 1506th of 2539
total : 1507th of 2539
total : 1508th of 2539
total : 1509th of 2539
total : 1510th of 2539
total : 1511th of 2539
total : 1512th of 2539
total : 1513th of 2539
total : 1514th of 2539
total : 1515th of 2539
total : 1516th of 2539
total : 1517th of 2539
total : 1518th of 2539
total : 1519th of 2539
total : 1520th of 2539
total : 1521th of 2539
total : 152

total : 1836th of 2539
total : 1837th of 2539
total : 1838th of 2539
total : 1839th of 2539
total : 1840th of 2539
total : 1841th of 2539
total : 1842th of 2539
total : 1843th of 2539
total : 1844th of 2539
total : 1845th of 2539
total : 1846th of 2539
total : 1847th of 2539
total : 1848th of 2539
total : 1849th of 2539
total : 1850th of 2539
total : 1851th of 2539
total : 1852th of 2539
total : 1853th of 2539
total : 1854th of 2539
total : 1855th of 2539
total : 1856th of 2539
total : 1857th of 2539
total : 1858th of 2539
total : 1859th of 2539
total : 1860th of 2539
total : 1861th of 2539
total : 1862th of 2539
total : 1863th of 2539
total : 1864th of 2539
total : 1865th of 2539
total : 1866th of 2539
total : 1867th of 2539
total : 1868th of 2539
total : 1869th of 2539
total : 1870th of 2539
total : 1871th of 2539
total : 1872th of 2539
total : 1873th of 2539
total : 1874th of 2539
total : 1875th of 2539
total : 1876th of 2539
total : 1877th of 2539
total : 1878th of 2539
total : 187

total : 2193th of 2539
total : 2194th of 2539
total : 2195th of 2539
total : 2196th of 2539
total : 2197th of 2539
total : 2198th of 2539
total : 2199th of 2539
total : 2200th of 2539
total : 2201th of 2539
total : 2202th of 2539
total : 2203th of 2539
total : 2204th of 2539
total : 2205th of 2539
total : 2206th of 2539
total : 2207th of 2539
total : 2208th of 2539
total : 2209th of 2539
total : 2210th of 2539
total : 2211th of 2539
total : 2212th of 2539
total : 2213th of 2539
total : 2214th of 2539
total : 2215th of 2539
total : 2216th of 2539
total : 2217th of 2539
total : 2218th of 2539
total : 2219th of 2539
total : 2220th of 2539
total : 2221th of 2539
total : 2222th of 2539
total : 2223th of 2539
total : 2224th of 2539
total : 2225th of 2539
total : 2226th of 2539
total : 2227th of 2539
total : 2228th of 2539
total : 2229th of 2539
total : 2230th of 2539
total : 2231th of 2539
total : 2232th of 2539
total : 2233th of 2539
total : 2234th of 2539
total : 2235th of 2539
total : 223

### kilt_dpr_nq_dev_pos 
    - is the result
    - `kilt_rand_nq_dev_pos1.json`

In [43]:
pprint(kilt_bm25_nq_dev_pos[0])

{'answers': ['the therefore sign',
             'therefore sign',
             'the therefore sign ( ∴ ) is generally used before a logical '
             'consequence , such as the conclusion of a syllogism',
             'a logical consequence , such as the conclusion of a syllogism'],
 'ctxs': [{'psg_id': 3894652,
           'text': 'Therefore sign In logical argument and mathematical proof, '
                   'the therefore sign (∴) is generally used before a logical '
                   'consequence, such as the conclusion of a syllogism. The '
                   'symbol consists of three dots placed in an upright '
                   'triangle and is read "therefore". It is encoded at . While '
                   'it is not generally used in formal writing, it is used in '
                   'mathematics and shorthand. It is complementary to . '
                   'History. According to Cajori, "A History of Mathematical '
                   'Notations," Johann Rahn used both t

In [44]:
pos = 1
filename = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos{pos}.json'

In [45]:
filename

'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos1.json'

In [46]:
instances = deepcopy(kilt_bm25_nq_dev_pos)

In [47]:
# utils.save_json(instances, filename)

## Swappings up to position 5
- Based on previous result, incrementally create new files with different position of positive ctx

In [48]:
def create_new_instance(instances, previous_pos, new_position):
    new_instance = deepcopy(instances)
    for ins in new_instance:
        ins['ctxs'][previous_pos-1], ins['ctxs'][new_position-1] = ins['ctxs'][new_position-1], ins['ctxs'][previous_pos-1]
    return new_instance

In [49]:
def checking_with_previous(instances, instances2, previous_pos, new_position):
    # checking
    for in1, in2 in zip(instances, instances2):
        if in1['id'] != in2['id']:
            print('not matching id')
            return False
        if in1['question'] != in2['question']:
            print('not mathcing question')
            return False
        if in1['answers'] != in2['answers']:
            print('not mathcing question')
            return False
        if in1['ctxs'][:previous_pos-1] + in1['ctxs'][previous_pos:] != in2['ctxs'][:new_position-1] + in2['ctxs'][new_position:]:
            print('negative not mathcing')
            pprint(in1['ctxs'])
            print('-----')
            pprint(in2['ctxs'])
            return False
        if in1['ctxs'][previous_pos-1] != in2['ctxs'][new_position-1]:
            print('positive not mathcing')
            print(in1['ctxs'][previous_pos-1])
            print(in2['ctxs'][new_position-1])
            return False
    return True
    

In [50]:
last_ins_testing = deepcopy(instances)
for i in range(1, 6):
    prev_pos = i
    next_pos = prev_pos + 1 if prev_pos < 5 else 1
    filename = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos{next_pos}.json'
    instance2 = create_new_instance(instances, prev_pos, next_pos)
    print(f'prev_pos : {prev_pos} next_pos : {next_pos}')
    
    if next_pos == 1 and checking_with_previous(instances, last_ins_testing, prev_pos, next_pos):
        print(f'checking the first one and the last instance')
    elif checking_with_previous(instances, instance2, prev_pos, next_pos):
        utils.save_json(instance2, filename)
        print(f'Saving {next_pos} instance')
        
    instances = deepcopy(instance2)
        

prev_pos : 1 next_pos : 2
Saving 2 instance
prev_pos : 2 next_pos : 3
Saving 3 instance
prev_pos : 3 next_pos : 4
Saving 4 instance
prev_pos : 4 next_pos : 5
Saving 5 instance
prev_pos : 5 next_pos : 1
checking the first one and the last instance


## ====== The End ======



In [ ]:
# # checking
# def check_pos(instances, instance2, previous_pos, new_position):
#     for in1, in2 in zip(instances, instance2):
#         if in1['id'] != in2['id']:
#             print('not matching id')
#             break
#         if in1['question'] != in2['question']:
#             print('not mathcing question')
#             break
#         if in1['answers'] != in2['answers']:
#             print('not mathcing question')
#             break
#         if in1['ctxs'][:previous_pos-1] + in1['ctxs'][previous_pos:] != in2['ctxs'][:new_position-1] + in2['ctxs'][new_position:]:
#             print('negative not mathcing')
#             pprint(in1['ctxs'][:previous_pos-1] + in1['ctxs'][previous_pos:])
#             pprint(in2['ctxs'][:new_position-1] + in2['ctxs'][new_position:])
#             break
#         if in1['ctxs'][previous_pos-1] != in2['ctxs'][new_position-1]:
#             print('positive not mathcing')
#             print(in1['ctxs'][previous_pos-1])
#             print(in2['ctxs'][new_position-1])
#             break
    

In [ ]:
def save_json(data, filename):
    with open(filename, 'w') as fp:
        json.dump(data, fp, ensure_ascii=True)

In [ ]:
instances = deepcopy(kilt_dpr_nq_dev_pos)

In [ ]:
filename1 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos1.json'

In [ ]:
save_json(instances, filename1)

In [ ]:
instance2 = deepcopy(instances)

In [ ]:
previous_pos = 1
new_position = 2

In [ ]:
for ins in instance2:
    ins['ctxs'][previous_pos-1], ins['ctxs'][new_position-1] = ins['ctxs'][new_position-1], ins['ctxs'][previous_pos-1]

In [ ]:
check_pos(instances, instance2, previous_pos, new_position)

In [ ]:
filename = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos2.json'

In [ ]:
save_json(instance2, filename)

In [ ]:
instance3 = deepcopy(instance2)

In [ ]:
previous_pos = 2
new_position = 3

In [ ]:
def get_next_instance(previous_pos, new_position, instance):
    for ins in instance:
        ins['ctxs'][previous_pos-1], ins['ctxs'][new_position-1] = ins['ctxs'][new_position-1], ins['ctxs'][previous_pos-1]
    return instance

In [ ]:
instance3 = get_next_instance(previous_pos, new_position, instance3)

In [ ]:
check_pos(instance2, instance3, previous_pos, new_position)

In [ ]:
filename = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos{new_position}.json'

In [ ]:
save_json(instance3, filename)

In [ ]:
instance4 = deepcopy(instance3)

In [ ]:
previous_pos = 3
new_position = 4

In [ ]:
instance4 = get_next_instance(previous_pos, new_position, instance4)

In [ ]:
check_pos(instance3, instance4, previous_pos, new_position)

In [ ]:
filename = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos{new_position}.json'

In [ ]:
save_json(instance4, filename)

In [ ]:
instance5 = deepcopy(instance4)

In [ ]:
previous_pos = 4
new_position = 5

In [ ]:
instance5 = get_next_instance(previous_pos, new_position, instance5)

In [ ]:
check_pos(instance4, instance5, previous_pos, new_position)

In [ ]:
filename = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos{new_position}.json'

In [ ]:
save_json(instance5, filename)

In [ ]:
previous_pos = 5
new_position = 1

In [ ]:
check_pos(instance5, instances, previous_pos, new_position)

## Checking on files

In [ ]:
# filename1 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos1.json'

In [ ]:
filename1 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos1.json'
filename2 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos2.json'
filename3 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos3.json'
filename4 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos4.json'
filename5 = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_RANDOM/kilt_rand_nq_dev_pos5.json'

In [ ]:
file1 = open_json(filename1)
file2 = open_json(filename2)
file3 = open_json(filename3)
file4 = open_json(filename4)
file5 = open_json(filename5)

In [ ]:
pprint(file1[-1])

In [ ]:
pprint(file2[-1])

In [ ]:
pprint(file3[-1])

In [ ]:
pprint(file4[-1])

In [ ]:
pprint(file5[-1])